# import  efficientnet  to wok offline

In [ ]:
import sys
sys.path.append('../input/efficientnet-keras-dataset/efficientnet_kaggle')
! pip install -e ../input/efficientnet-keras-dataset/efficientnet_kaggle

In [ ]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers



import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
import time
import json
import copy
import gc
import os
import time
import random
from datetime import datetime

from PIL import Image
from tqdm.notebook import tqdm
from sklearn import model_selection, metrics
print("Tensorflow version " + tf.__version__)

In [ ]:
import efficientnet.keras as efn 


# tpu detaction and config

ref  https://www.tensorflow.org/guide/tpu   https://www.kaggle.com/docs/tpu

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

using gcs_path

In [ ]:

AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH =  KaggleDatasets().get_gcs_path('ranzcr-clip-catheter-line-classification')
IMAGENET_MEAN = tf.constant([0.485, 0.456, 0.406], dtype=tf.float32)
IMAGENET_STD = tf.constant([0.229, 0.224, 0.225], dtype=tf.float32)
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = 600
EPOCHS = 25

# working with tfrecoreds

In [ ]:
#just exploraing the recoreds to understand them schema
raw_dataset = tf.data.TFRecordDataset(GCS_PATH+'/train_tfrecords/00-1881.tfrec')

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) 
    image = tf.image.resize(image, (IMAGE_SIZE , IMAGE_SIZE))
    return image

In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
     
      "CVC - Abnormal": tf.io.FixedLenFeature([], tf.int64), 
    
    "CVC - Borderline": tf.io.FixedLenFeature([], tf.int64), 
    "CVC - Normal": tf.io.FixedLenFeature([], tf.int64), 
    "ETT - Abnormal": tf.io.FixedLenFeature([], tf.int64), 
    "ETT - Borderline": tf.io.FixedLenFeature([], tf.int64), 
    "ETT - Normal": tf.io.FixedLenFeature([], tf.int64), 
    "NGT - Abnormal": tf.io.FixedLenFeature([], tf.int64), 
    "NGT - Borderline": tf.io.FixedLenFeature([], tf.int64), 
    
    
        "NGT - Incompletely Imaged": tf.io.FixedLenFeature([], tf.int64), 

        "NGT - Normal": tf.io.FixedLenFeature([], tf.int64), 

        "Swan Ganz Catheter Present": tf.io.FixedLenFeature([], tf.int64), 

    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = [example['CVC - Abnormal'],example['CVC - Borderline'],example['CVC - Normal'],example['ETT - Abnormal'],example['ETT - Borderline'],example['ETT - Normal'],
                    example['NGT - Abnormal'],example['NGT - Borderline'],example['NGT - Incompletely Imaged'],example['NGT - Normal'],example['Swan Ganz Catheter Present']]
   
        return image, label
    idnum = example['image_name']
    return image, idnum

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset

In [ ]:
TRAINING_FILENAMES, VALID_FILENAMES = train_test_split(
    tf.io.gfile.glob(GCS_PATH + '/train_tfrecords/*.tfrec'),
    test_size=0.2, random_state=5
)

TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test_tfrecords/*.tfrec')

# doing random data augmention on tpu

In [ ]:
def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO) statement in the following function this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part of the TPU while the TPU itself is computing gradients.

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)   
    image = tf.image.random_contrast(image, 0.2, 0.5)
    image = tf.image.random_brightness(image , 0.2)
    image /= 255.0
    image = (image - IMAGENET_MEAN) / IMAGENET_STD    
    return image, label

In [ ]:
def data_augmentval(image, label):

    image /= 255.0
    image = (image - IMAGENET_MEAN) / IMAGENET_STD    
    return image, label

# datasets

In [ ]:
def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE) 
    dataset = dataset.repeat()
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALID_FILENAMES, labeled=True, ordered=ordered) 
    dataset = dataset.map(data_augmentval, num_parallel_calls=AUTOTUNE) 
    dataset = dataset.repeat()    
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALID_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)

print('Dataset: {} training images, {} validation images, {} (unlabeled) test images'.format(
    NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

# learning rate schudler

In [ ]:
def lrfn(epoch, bs=BATCH_SIZE, epochs=EPOCHS):
    # Config
    LR_START = 1e-6
    LR_MAX = 2e-4
    LR_FINAL = 1e-6
    LR_RAMPUP_EPOCHS = 4
    LR_SUSTAIN_EPOCHS = 0
    DECAY_EPOCHS = epochs  - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1
    LR_EXP_DECAY = (LR_FINAL / LR_MAX) ** (1 / (EPOCHS - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1))

    if epoch < LR_RAMPUP_EPOCHS: # exponential warmup
        lr = LR_START + (LR_MAX + LR_START) * (epoch / LR_RAMPUP_EPOCHS) ** 2.5
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS: # sustain lr
        lr = LR_MAX
    else: # cosine decay
        epoch_diff = epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS
        decay_factor = (epoch_diff / DECAY_EPOCHS) * math.pi
        decay_factor= (tf.math.cos(decay_factor).numpy() + 1) / 2        
        lr = LR_FINAL + (LR_MAX - LR_FINAL) * decay_factor

    return lr

In [ ]:
# plots the learning rate schedule
def show_lr_schedule(bs=BATCH_SIZE, epochs=EPOCHS):
    rng = [i for i in range(epochs)]
    y = [lrfn(x, bs=bs, epochs=epochs) for x in rng]
    x = np.arange(epochs)
    x_axis_labels = list(map(str, np.arange(1, epochs+1)))
    print('init lr {:.1e} to {:.1e} final {:.1e}'.format(y[0], max(y), y[-1]))
    
    plt.figure(figsize=(30, 10))
    plt.xticks(x, x_axis_labels, fontsize=16) # set tick step to 1 and let x axis start at 1
    plt.yticks(fontsize=16)
    plt.plot(rng, y)
    plt.grid()
    plt.show()
    
show_lr_schedule()

In [ ]:
layers = tf.keras.layers
models = tf.keras.models
losses = tf.keras.losses
optimizers = tf.keras.optimizers 
metrics = tf.keras.metrics
utils = tf.keras.utils
callbacks = tf.keras.callbacks

# model creation function

In [ ]:
def create_model(met,trin=0.35):

    base =  efn.EfficientNetB7(
        include_top=False,
        weights="imagenet",
        
        input_shape= None , # (IMAGE_SIZE , IMAGE_SIZE, 3),
       
    )
    trin = round(len(base.layers)*trin)
    for layer in base.layers[:-trin]:
        layer.trainable = False
    model = models.Sequential()
    '''auglayer = tf.keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                       layers.experimental.preprocessing.RandomRotation(0.2),
                        layers.experimental.preprocessing.RandomContrast(1),
                        layers.experimental.preprocessing.RandomZoom(0.2)])
    model.add(keras.Input(shape=(512,512,3,)))
    model.add(auglayer) 
    '''
    model.add(base)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.4))


    model.add(layers.Dense(512, activation='relu'))

    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(128, activation='relu'))

    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(11, activation='sigmoid'))

    #adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile( optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',  
        metrics=met)
    return model

# defin the model within tpu stratgy and set parameters

In [ ]:
with strategy.scope():
    METRICS = [ keras.metrics.AUC(multi_label=True)
           
    ]
    model = create_model(met=METRICS,trin=0.75)
model.summary()

In [ ]:
for layer in model.layers:
    print(layer.get_output_at(0).get_shape().as_list())

In [ ]:
# load data
train_dataset = get_training_dataset()
valid_dataset = get_validation_dataset()

In [ ]:
train_dataset

In [ ]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch, epochs=EPOCHS), verbose=1)

In [ ]:
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALID_STEPS = NUM_VALIDATION_IMAGES // BATCH_SIZE
#checkpoits for saving best wights based on val_loss 
mcp_save = callbacks.ModelCheckpoint('mamoncheckp600tpu-efnnet-halffreez-schdl.hdf5', save_best_only=True, monitor='val_loss', mode='min')

#start training while history will contain metrices
history = model.fit(train_dataset, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=EPOCHS,
                    validation_data=valid_dataset,
                    validation_steps=VALID_STEPS ,callbacks=[mcp_save , lr_callback] )# , class_weight=class_weight)

In [ ]:
# print out variables available to us  
print(history.history.keys())

In [ ]:
# create learning curves to evaluate model performance
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['auc', 'val_auc']].plot();

In [ ]:
model2 = keras.models.load_model('./mamoncheckp600tpu-efnnet-halffreez-schdl.hdf5')
print("Evaluate on test data")
results = model2.evaluate(valid_dataset, steps=VALID_STEPS)
print("test loss, test acc:", results)

In [ ]:
model2.save('efnetb7-acc9218.hdf5')
